(sec-qiskit)=
# Qiskit


Qiskit is an open-source software development kit (SDK) for quantum computation. It runs inside Python platform.

QIskit provides a large set of tools for

1. developing new quantum algorithms and exploring new idea
2. constructing a quantum circuit and testing it by running simulation on a classical computer
3. executing the circuit on a real quantum computer through IBM Quantum Experience.

We will use QIskit for all these three important coding steps.

You can find useful information about Qiskit including tutorials and API documentation at [qiskit.org](https://qiskit.org).

## Installation

It is a set of python libraries but not included in Anaconda.  We need to install them manually.

```
pip install qiskit
pip install qiskit[visualization]
```

Since conda does not manage these packages, you must update the package when a new version becomes available.  To check the current version, run the following command in the anaconda terminal window.

On MS Windows, use Anaconda Powershell Prompt.
```
pip list | select-string "qiskit"

qiskit                            0.36.2
qiskit-aer                        0.10.4
qiskit-ibmq-provider              0.19.1
qiskit-ignis                      0.7.1
qiskit-terra                      0.20.2
```

On Linux
```
pip list | grep qiskit

qiskit                            0.36.2
qiskit-aer                        0.10.4
qiskit-ibmq-provider              0.19.1
qiskit-ignis                      0.7.1
qiskit-terra                      0.20.2
```

To check if updates are available, the following command shows newer versions.

```
# On MS Windows
pip list --outdated | select-string "qiskit"

# On Linux
pip list --outdated | grep qiskit
```



## IBM Quantum Experience

In order to take the full advantage of Qiskit, you must first create an IBM Quantum Experience account.  With IBMid, you can run Qiskit codes on real IBM quantum computers as well as on realistic simulations on your computer.  Go to
[quantum-computing.ibm.com](https://quantum-computing.ibm.com/) and set up an account.
Log in to your account and take a look at IBM Quantum Dashboard where you find many useful stuffs which we discuss in later chapters.


## API key

Next, you need to obtain an API key and save it in a local computer.

1. Log in to IBM Quantum Experience at [quantum-computing.ibm.com](https://quantum-computing.ibm.com/)
2. Click the user icon at the upper-right corner.
3. Click "Account setting".
4. Click "Generate new token"
5. Click copy icon at the right end of the token box.  Your token is copied to the clipboard.
6. Open a text editor and paste the token.  Save it it to a temporary file so that you can copy the token at a later time if needed. Delete the file after the key is properly installed.
7. Open an Anaconda terminal window.
8. Start python and execute the following command at the python prompt:

```
>>> from qiskit import IBMQ
>>> IBMQ.save_account('past your token here')
```
The token must be inside the single quotes.  Now, we verify if the token works.

```
>>> IBMQ.load_account()
```
You should get the following response:
```
<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>
```
If it worked, delete the temporary file created at step 6.  Otherwise, something went wrong. Try step 8 again.  Make it sure that the whole key is pasted.

If you work on multiple computers, you have to install the same API on each machine.

## Using Qiskit

Since Qiskit is a collection of python modules, we must import it to your code before using it.  The package is so large that importing the entire package is not a good idea. In this book, we use only a small portion of it.  As you move on, this book introduces some basic modules absolutely necessary for quantum computing and explains how to use them step by step.

## Suggested Reading

As mentioned above, there are various online resources at [qiskit.org](https://qiskit.org) and [quantum-computing.ibm.com](https://quantum-computing.ibm.com/). In particular, the following online textbook is recommended.

* [Learn Quantum Computation using Qiskit](https://qiskit.org/textbook/)

In addition, the following paperback book is recommended.

* H. Norl&eacute;n: [*Quantum Computing in Practice with Qiskit and IBM Quantum Experience*](https://www.packtpub.com/product/quantum-computing-in-practice-with-qiskit-and-ibm-quantum-experience/9781838828448) (Packt, 2020).
Source codes can be obtained at [github](https://github.com/PacktPublishing/Quantum-Computing-in-Practice-with-Qiskit-and-IBM-Quantum-Experience).

In [1]:
# Common imports
import numpy as np

# Qiskit imports (for illustrative purposes; in practice install and import Qiskit)
# from qiskit import QuantumCircuit, Aer, execute
# from qiskit.quantum_info import Statevector, Operator

# Define classical payoff parameters (example PD payoffs)
R, P, T, S = 3, 1, 5, 0

def classical_payoff_PD(choice_A, choice_B):
    """
    Compute classical Prisoner's Dilemma payoff for players.
    choice_A, choice_B: 0 for Cooperate, 1 for Defect.
    Returns (payoff_A, payoff_B).
    """
    if choice_A == 0 and choice_B == 0:  # C,C
        return (R, R)
    elif choice_A == 0 and choice_B == 1:  # C,D
        return (S, T)
    elif choice_A == 1 and choice_B == 0:  # D,C
        return (T, S)
    else:  # D,D
        return (P, P)

def classical_payoff_minority(choices):
    """
    Classical Minority game payoff for a list of N binary choices.
    Each minority player gets +1, others get 0.
    choices: list of 0/1 choices for all players.
    Returns a list of payoffs (same length as choices).
    """
    # Count how many chose 0 and 1
    count0 = choices.count(0)
    count1 = choices.count(1)
    payoffs = [0]*len(choices)
    if count0 == 0 or count1 == 0:
        # No minority if all chose same; everybody loses (0 payoff)
        return payoffs
    # Determine minority side (0 or 1)
    minority_side = 0 if count0 < count1 else 1
    # Award 1 to each player on minority side
    for i, c in enumerate(choices):
        if c == minority_side:
            payoffs[i] = 1
    return payoffs

# Example usage (classical games)
print("PD payoffs (C,C):", classical_payoff_PD(0,0))  # expect (R,R) = (3,3)
print("PD payoffs (C,D):", classical_payoff_PD(0,1))  # expect (S,T) = (0,5)
print("Minority payoffs [0,0,1]:", classical_payoff_minority([0,0,1]))  # [0,0,1] -> 1 for last player

PD payoffs (C,C): (3, 3)
PD payoffs (C,D): (0, 5)
Minority payoffs [0,0,1]: [0, 0, 1]


In [2]:
import scipy.linalg
# Pauli matrices and identity
sigma_x = np.array([[0,1],[1,0]], dtype=complex)
sigma_y = np.array([[0,-1j],[1j,0]], dtype=complex)
sigma_z = np.array([[1,0],[0,-1]], dtype=complex)
I = np.eye(2, dtype=complex)

def entangling_J(gamma):
    """
    Construct the 4x4 entangling operator J(gamma) = exp(i * gamma * sigma_x ⊗ sigma_x / 2).
    """
    D = sigma_x
    return scipy.linalg.expm(1j * gamma * np.kron(D, D) / 2)

def strategy_U(theta, phi):
    """
    Single-qubit unitary U(θ, φ) as defined in Eisert et al. (for PD).
    """
    return np.array(
        [[np.cos(theta/2),                -np.exp(1j*phi)*np.sin(theta/2)],
         [np.exp(-1j*phi)*np.sin(theta/2), np.cos(theta/2)]
    ], dtype=complex)

def quantum_payoff_PD(theta_A, phi_A, theta_B, phi_B, gamma):
    """
    Simulate quantum Prisoner's Dilemma for given strategy angles and entanglement.
    Returns (E[Payoff_A], E[Payoff_B]).
    """
    # Initial state |00>
    psi0 = np.array([1,0,0,0], dtype=complex)
    # Prepare entangled state J |00>
    J = entangling_J(gamma)
    psi = J @ psi0
    # Apply local unitaries U_A ⊗ U_B
    U_A = strategy_U(theta_A, phi_A)
    U_B = strategy_U(theta_B, phi_B)
    psi = np.kron(U_A, U_B) @ psi
    # Disentangle with J^†
    psi = J.conj().T @ psi
    # Compute outcome probabilities
    probs = np.abs(psi)**2  # order [00, 01, 10, 11]
    # Compute expected payoffs
    payoff_A = (probs[0]*R + probs[1]*S + probs[2]*T + probs[3]*P)
    payoff_B = (probs[0]*R + probs[1]*T + probs[2]*S + probs[3]*P)
    return payoff_A, payoff_B

# Example: classical strategies (C=I, D=iX)
theta_C, phi_C = 0, 0   # U(0,0) = I
theta_D, phi_D = np.pi, 0  # U(pi,0) = iσ_x (up to phase)
print("Quantum PD (C,C,γ=0):", quantum_payoff_PD(theta_C,phi_C,theta_C,phi_C, gamma=0))
print("Quantum PD (C,D,γ=0):", quantum_payoff_PD(theta_C,phi_C,theta_D,phi_D, gamma=0))

Quantum PD (C,C,γ=0): (np.float64(3.0), np.float64(3.0))
Quantum PD (C,D,γ=0): (np.float64(1.1248198369963932e-32), np.float64(5.0))


In [3]:
import itertools

def ghz_state(gamma):
    """
    Return a 3-qubit entangled state vector parameterized by gamma.
    For gamma = π/2, it's the GHZ (|000>+|111>)/√2. For simplicity, here we assume
    maximal entanglement.
    """
    # For gamma=π/2, use (|000> + |111>)/sqrt(2)
    return np.array([1,0,0,0,  0,0,0,1], dtype=complex) / np.sqrt(2)

def quantum_payoff_minority(strategies, gamma):
    """
    Simulate quantum Minority game for 3 players.
    strategies: list of 3 choices, each in {0,1}, encoded as Pauli-X flips if 1.
    gamma: entanglement parameter (here we assume either 0 or π/2).
    Returns payoffs list for the 3 players.
    """
    # Initial state
    if gamma == 0:
        # separable state |000>
        psi = np.zeros(8, dtype=complex); psi[0] = 1
    else:
        psi = ghz_state(gamma)
    # Apply strategies: here model classical choices as X gates if choice=1
    U_total = np.eye(8, dtype=complex)
    for i, choice in enumerate(strategies):
        if choice == 1:
            # Apply X on qubit i: build operator I^⊗i ⊗ X ⊗ I^(rest)
            op = 1
            for q in range(3):
                op = np.kron(op, sigma_x if q==i else I)
            U_total = op @ U_total
    psi = U_total @ psi
    # Measurement probabilities
    probs = np.abs(psi)**2  # length 8 array for outcomes 000,001,...,111
    # Determine minority outcome for each basis state
    payoffs = [0,0,0]
    # Loop over all basis outcomes
    for idx, p in enumerate(probs):
        bits = [(idx>>2)&1, (idx>>1)&1, idx&1]  # convert index to bitstring
        payoff_round = classical_payoff_minority(bits)
        # Expected payoff adds with weight p
        for i in range(3):
            payoffs[i] += p * payoff_round[i]
    return payoffs

# Example: each player chooses classically [C,C,D] => [0,0,1]
print("Quantum Minority (0,0,1), no entanglement:", quantum_payoff_minority([0,0,1], gamma=0))
print("Quantum Minority (0,0,1), entangled GHZ:", quantum_payoff_minority([0,0,1], gamma=np.pi/2))

Quantum Minority (0,0,1), no entanglement: [np.float64(0.0), np.float64(0.0), np.float64(1.0)]
Quantum Minority (0,0,1), entangled GHZ: [np.float64(0.0), np.float64(0.0), np.float64(0.9999999999999998)]


In [4]:
!pip install qiskit
!pip install qiskit[visualization]
!pip install qiskit-aer

from qiskit.providers.aer.noise import pauli_error, amplitude_damping_error, NoiseModel

# Example: one-qubit depolarizing channel with probability p
def get_depolarizing_noise(p):
    error = pauli_error([('X', p/3), ('Y', p/3), ('Z', p/3), ('I', 1-p)])
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error, ['u1','u2','u3'])
    return noise_model

# To apply noise, we would run QasmSimulator with the noise model:
# backend = Aer.get_backend('qasm_simulator')
# noisy_result = execute(circuit, backend, noise_model=noise_model, shots=1024).result()

ModuleNotFoundError: No module named 'qiskit.providers.aer'

In [5]:
from qiskit.providers.aer.noise import pauli_error, amplitude_damping_error, NoiseModel
print("Successfully imported qiskit.providers.aer.noise")

ModuleNotFoundError: No module named 'qiskit.providers.aer'